# Train the model

In [ ]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import scipy

LOGDIR = r'C:\Users\sweta\OneDrive\Desktop\DL Projects\Self driving cars\mine\save'
sess = tf.compat.v1.InteractiveSession()

#L2NormConst = 0.001
train_vars = tf.compat.v1.trainable_variables() #it will return all the variables. Here, all the weights and biases are variables which
#are trainable.

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) #* L2NormConst
#since this is a regression problem so above loss is mean-squared-error loss
train_step = tf.compat.v1.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.compat.v1.initialize_all_variables())

# create a summary to monitor cost tensor
tf.compat.v1.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.compat.v1.summary.merge_all()

saver = tf.compat.v1.train.Saver(write_version = saver_pb2.SaverDef.V2)


# op to write logs to Tensorboard
logs_path = r'C:\Users\sweta\OneDrive\Desktop\DL Projects\Self driving cars\mine\log'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.compat.v1.get_default_graph())

epochs = 30
batch_size = 100

for epoch in range(epochs):    
  for i in range(int(num_images/batch_size)):
    xs, ys = LoadTrainBatch(batch_size)
    train_step.run(feed_dict={x: xs, y_: ys, keep_prob: 0.8})
    if i % 10 == 0:
      xs, ys = LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)
    
    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

